# Tutorial 03: Writing first EMNIST Program
By Rahul GAWAI

## Introduction
This program is inspired by [GitHub](https://github.com/Hvass-Labs/TensorFlow-Tutorials)

Python used: 3.6
TensorFlow version: 1.10

## Importing packages

In [29]:
import tensorflow as tf

import csv
import sys
import os
import urllib.request
import gzip
import numpy as np
import matplotlib.pyplot as plt
import random
import cv2


In [4]:
img_w = 28
img_h = 28
img_shape = (img_w, img_h)
img_shape_storage = (img_w, img_h, 1)

TRAIN_VAL_RATIO = 80

DATASET_DIR = '/home/rahul/ML/SSD/emnist/'
DATASET_FILES_DIR = '/home/rahul/ML/SSD/emnist/emnist/'

dataset_index_label = '/home/rahul/ML/SSD/emnist/dataset_index_label.csv'
dataset_fname_index = '/home/rahul/ML/SSD/emnist/dataset_fname_index.csv'
dataset_fname_index_shuffle = '/home/rahul/ML/SSD/emnist/dataset_fname_index_shuffle.csv'
dataset_fname_index_train = '/home/rahul/ML/SSD/emnist/dataset_fname_index_train.csv'
dataset_fname_index_val = '/home/rahul/ML/SSD/emnist/dataset_fname_index_val.csv'

training_recname = '%s/emnist_%s.tfrecord' % (DATASET_DIR, 'dataset_fname_index_train')
testing_recname = '%s/emnist_%s.tfrecord' % (DATASET_DIR, 'dataset_fname_index_val')

export_dir = os.path.join('./02_QAT_MNIST/', 'export')

In [5]:
index = 0
count = 0
label = []

indexFile = open(dataset_index_label, 'w', encoding="utf-8")
fnameFile = open(dataset_fname_index, 'w', encoding="utf-8")

for sdir in os.listdir(DATASET_FILES_DIR):
    indexFile.write(sdir+"\n")
    label.append(sdir)
    for file in os.listdir(DATASET_FILES_DIR + sdir):
        row = DATASET_FILES_DIR+sdir+'/'+file + "," + str(index) + "\n"
        fnameFile.write(row)
        count = count + 1
    index = index + 1
           
indexFile.close()
fnameFile.close()

print("Added Files: ", count, " and labels: ", label, index)

fnameFile = open(dataset_fname_index, 'r', encoding="utf-8")
fnameFileShuffle = open(dataset_fname_index_shuffle, 'w', encoding="utf-8")
fnameTrain = open(dataset_fname_index_train, 'w', encoding="utf-8")
fnameVal = open(dataset_fname_index_val, 'w', encoding="utf-8")

lis=[line.split() for line in fnameFile]
random.shuffle(lis)

writer = csv.writer(fnameFileShuffle)

for item in lis:
    row = item[0].split(",")
    rowsdtr = row[0] + "," + row[1] + "\n"
    fnameFileShuffle.write(rowsdtr)

total = len(lis)
TrainingEnd = int((total*TRAIN_VAL_RATIO)/100)

for count in range(0, TrainingEnd):
    item = lis[count]
    row = item[0].split(",")
    rowsdtr = row[0] + "," + row[1] + "\n"
    fnameTrain.write(rowsdtr)

for count in range(TrainingEnd, total):
    item = lis[count]
    row = item[0].split(",")
    rowsdtr = row[0] + "," + row[1] + "\n"
    fnameVal.write(rowsdtr)

fnameFileShuffle.close()
fnameFile.close()

print("Training File: ", TrainingEnd, "entries.")
print("Evaluation File: ", total-TrainingEnd, "entries.")
print("Files are Ready !!")

Added Files:  372451  and labels:  ['O', 'R', 'Q', 'C', 'U', 'X', 'F', 'B', 'A', 'M', 'H', 'P', 'J', 'K', 'I', 'T', 'Y', 'E', 'L', 'V', 'Z', 'D', 'N', 'S', 'G', 'W'] 26
Training File:  297960 entries.
Evaluation File:  74491 entries.
Files are Ready !!


## Download dataset to a local folder.

In [6]:
def plot_images(images, cls_true, cls_pred=None, img_w=28, img_h=28):
    assert len(images) == len(cls_true) == 9


    # Create figure with 3x3 sub-plots.
    fig, axes = plt.subplots(3, 3)
    fig.subplots_adjust(hspace=0.3, wspace=0.3)

    for i, ax in enumerate(axes.flat):
        # Plot image.
        ax.imshow(images[i].reshape(img_shape), cmap='binary')

        # Show true and predicted classes.
        if cls_pred is None:
            xlabel = "True: {0}".format(cls_true[i])
        else:
            xlabel = "True: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

        ax.set_xlabel(xlabel)
        
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])
        
    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

In [7]:
# Get the first images from the test-set.
# images = X_test[0:9]

# Get the true classes for those images.
# cls_true = Y_test_cls[0:9]

# Plot the images and labels using our helper-function above.
# plot_images(images=images, cls_true=cls_true)

## One Hot Encoding

In [8]:
# y_train_cls_char = Y_train_cls.astype(np.int)
# y_val_cls_char = Y_val_cls.astype(np.int)
# y_test_cls_char = Y_test_cls.astype(np.int)

# Y_train_OHE = np.eye(10, dtype=float)[y_train_cls_char]
# Y_val_OHE = np.eye(10, dtype=float)[y_val_cls_char]
# Y_test_OHE = np.eye(10, dtype=float)[y_test_cls_char]

# Y_test_OHE[0:5, :]

## TF-record (Optional)

[Referred this implementation to create tf.record](https://github.com/tensorflow/models/blob/master/research/slim/datasets/download_and_convert_mnist.py)

In [9]:
def int64_feature(values):
  if not isinstance(values, (tuple, list)):
    values = [values]
  return tf.train.Feature(int64_list=tf.train.Int64List(value=values))

In [10]:
def image_to_tfexample(image_data, image_format, height, width, class_id):
  return tf.train.Example(features=tf.train.Features(feature={
      'image/encoded':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_data])),
      'image/format':  tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_format])),
      'image/class/label': int64_feature(class_id),
      'image/height': int64_feature(height),
      'image/width': int64_feature(width)
  }))

In [32]:
def _add_to_tfrecord(num_images, tfrecord_writer, dataList):
    with tf.Graph().as_default():
        image = tf.placeholder(dtype=tf.uint8, shape=img_shape_storage)
        encoded_png = tf.image.encode_png(image)

        num_images = len(dataList)
#         images = images.reshape(-1, img_w, img_h, 1)

        with tf.Session('') as sess:
            for j in range(num_images):
                if (j % 1000) == 0:
                    sys.stdout.write('\n>> Converting image %d/%d %s' % (j + 1, num_images, (dataList[j][0])))
                    sys.stdout.flush()

                pngFile=open(dataList[j][0],'rb')
                png_string1 = pngFile.read();
       
                x = np.frombuffer(png_string1, dtype='uint8')
                sample = cv2.imdecode(x, cv2.IMREAD_UNCHANGED).reshape(28,28,1);
                pngFile.close()
                
#                 sys.stdout.write('>> num_images %s ' % pngFile )
#                 sys.stdout.write('>> num_images %s ' % dataList[j][0])
#                 sys.stdout.write('>> num_images %d ' % int(dataList[j][1]))
#                 sys.stdout.flush()
#                 sample = tf.cast(tf.image.decode_png(png_string1, channels=1), tf.uint8)

                png_string = sess.run(encoded_png, feed_dict={image: sample})
                example = image_to_tfexample(png_string1, 'png'.encode(), img_w, img_h, int(dataList[j][1]))
                tfrecord_writer.write(example.SerializeToString())

        sys.stdout.write('\r>> Converting image %d/%d' % (num_images, num_images))
        return

#                 if dump == True:
#                     pngFname = target + "/PNGs/" + "train_%d.png" % (j)
#                     pngFile=open(pngFname,'wb')
#                     pngFile.write(png_string);
#                     pngFile.close()         


In [ ]:
fnameTrain = open(dataset_fname_index_train, 'r', encoding="utf-8")
fnameVal = open(dataset_fname_index_val, 'r', encoding="utf-8")

TrainList=[line.split(",") for line in fnameTrain]
TrainListLen = len(TrainList)
print("[Train Set Length: ", TrainListLen, " entries.]")

if os.path.exists(training_recname):
    with tf.python_io.TFRecordWriter(training_recname) as tfrecord_writer:
        _add_to_tfrecord(TrainListLen, tfrecord_writer, TrainList)
        
ValList=[line.split(",") for line in fnameVal]
ValListLen = len(ValList)
print("\n[ Val Set Length: ", ValListLen, " entries.]")

if os.path.exists(testing_recname):
    with tf.python_io.TFRecordWriter(testing_recname) as tfrecord_writer:
        _add_to_tfrecord(ValListLen, tfrecord_writer, ValList)
        
print("\n[ TFRecords creation is complete!]")

[Train Set Length:  297960  entries.]

>> Converting image 1/297960 /home/rahul/ML/SSD/emnist/emnist/I/I-997.png
>> Converting image 1001/297960 /home/rahul/ML/SSD/emnist/emnist/C/C-15092.png
>> Converting image 2001/297960 /home/rahul/ML/SSD/emnist/emnist/M/M-10163.png
>> Converting image 3001/297960 /home/rahul/ML/SSD/emnist/emnist/N/N-7638.png
>> Converting image 4001/297960 /home/rahul/ML/SSD/emnist/emnist/S/S-28431.png
>> Converting image 5001/297960 /home/rahul/ML/SSD/emnist/emnist/S/S-29642.png
>> Converting image 6001/297960 /home/rahul/ML/SSD/emnist/emnist/O/O-50475.png
>> Converting image 7001/297960 /home/rahul/ML/SSD/emnist/emnist/A/A-9462.png
>> Converting image 8001/297960 /home/rahul/ML/SSD/emnist/emnist/B/B-7650.png
>> Converting image 9001/297960 /home/rahul/ML/SSD/emnist/emnist/O/O-44071.png
>> Converting image 10001/297960 /home/rahul/ML/SSD/emnist/emnist/T/T-4075.png

## Main Function()

In [172]:
def parser(record):
    keys_to_features = {
        "image/encoded":  tf.FixedLenFeature([], tf.string),
        "image/format":  tf.FixedLenFeature([], tf.string),
        "image/class/label": tf.FixedLenFeature([], tf.int64),
        "image/height": tf.FixedLenFeature([], tf.int64),
        "image/width": tf.FixedLenFeature([], tf.int64)
    }

    parsed = tf.parse_single_example(record, keys_to_features)

    height = tf.cast(parsed["image/height"], tf.int32)
    width = tf.cast(parsed["image/width"], tf.int32)
    label = tf.cast(parsed["image/class/label"], tf.int32)
    image = tf.cast(tf.image.decode_png(parsed["image/encoded"], channels=1), tf.float32)

#     sys.stdout.write('>> height %d ' % height)
#     sys.stdout.flush()

 
    return {'image': image}, label

In [173]:
def input_fn(filenames):
    
    BATCH_SIZE = 64
    THREADS = 4
    PREFETCH = 64
    
    dataset = tf.data.TFRecordDataset(filenames=filenames, num_parallel_reads=THREADS)
    dataset = dataset.apply(
        tf.contrib.data.shuffle_and_repeat(1024, 1)
    )
    dataset = dataset.apply(
        tf.contrib.data.map_and_batch(parser, BATCH_SIZE)
    )

    dataset = dataset.prefetch(buffer_size=PREFETCH)
    return dataset

In [174]:
def train_input_fn():
    return input_fn(filenames=[training_recname])

In [175]:
def val_input_fn():
    return input_fn(filenames=[testing_recname])

In [176]:
def cnn_model_fn(features, labels, mode, params):
    """Model function for CNN."""
    # Input Layer
    # Reshape X to 4-D tensor: [batch_size, width, height, channels]
    # MNIST images are 28x28 pixels, and have one color channel

    # Print all of the operations in the default graph.

    net = features["image"]

    net = tf.identity(net, name="input_tensor")

    net = tf.reshape(net, [-1, 28, 28, 1])

    # Convolutional Layer #1
    # Computes 32 features using a 5x5 filter with ReLU activation.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 28, 28, 1]
    # Output Tensor Shape: [batch_size, 28, 28, 32]
    conv1 = tf.layers.conv2d(inputs=net, filters=32,kernel_size=[5, 5],padding="same", activation=tf.nn.relu, name="conv1-layer")
    
    # Pooling Layer #1
    # First max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 28, 28, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 32]
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2, name="pool1-layer")

    # Convolutional Layer #2
    # Computes 64 features using a 5x5 filter.
    # Padding is added to preserve width and height.
    # Input Tensor Shape: [batch_size, 14, 14, 32]
    # Output Tensor Shape: [batch_size, 14, 14, 64]
    conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu, name="conv2-layer")

    # Pooling Layer #2
    # Second max pooling layer with a 2x2 filter and stride of 2
    # Input Tensor Shape: [batch_size, 14, 14, 64]
    # Output Tensor Shape: [batch_size, 7, 7, 64]
    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2, name="pool2-layer")

    # Flatten tensor into a batch of vectors
    # Input Tensor Shape: [batch_size, 7, 7, 64]
    # Output Tensor Shape: [batch_size, 7 * 7 * 64]
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])

    # Dense Layer
    # Densely connected layer with 1024 neurons
    # Input Tensor Shape: [batch_size, 7 * 7 * 64]
    # Output Tensor Shape: [batch_size, 1024]
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu, name="dense-layer")

    # Add dropout operation; 0.6 probability that element will be kept
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits layer
    # Input Tensor Shape: [batch_size, 1024]
    # Output Tensor Shape: [batch_size, 10]
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        "classes": tf.argmax(input=logits, axis=1), "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

#     if mode == tf.estimator.ModeKeys.TRAIN:
#         # Call rewriter to produce graph with fake quant ops and folded batch norms
#         # quant_delay delays start of quantization till quant_delay steps, allowing
#         # for better model accuracy.
#         g = tf.get_default_graph()
#         tf.contrib.quantize.create_training_graph(input_graph=g, quant_delay=1000)

#     if mode == tf.estimator.ModeKeys.EVAL:
#         # Call the eval rewrite which rewrites the graph in-place with
#         # FakeQuantization nodes and fold batchnorm for eval.
#         g = tf.get_default_graph()
#         tf.contrib.quantize.create_eval_graph(input_graph=g)     


    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }

    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [177]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

if os.path.exists("03_QAT_EMNIST_DEEP"):
    tf.gfile.DeleteRecursively("03_QAT_EMNIST_DEEP");tf.gfile.MakeDirs("03_QAT_EMNIST_DEEP")
    tf.summary.FileWriterCache.clear()

model = tf.estimator.Estimator(model_fn=cnn_model_fn,
                               params={"learning_rate": 1e-4},
                               model_dir="03_QAT_EMNIST_DEEP")

import timeit

EPOCHS = 5
STEP_SIZE = 10
count = 0

while (count < EPOCHS):
    start_time = timeit.default_timer()
    model.train(input_fn=train_input_fn, steps=STEP_SIZE)
    elapsed = timeit.default_timer() - start_time
    sys.stdout.write("Time Take per Iternation of training is : {0:%}".format(elapsed))

    result = model.evaluate(input_fn=val_input_fn)
    #print(result)
    sys.stdout.write("Classification accuracy: {0:.2%}".format(result["accuracy"]))
    sys.stdout.flush()
    count = count + 1

# print("conv1-layer/bias", model.get_variable_value('conv1-layer/act_quant/conv1-layer/act_quant/min/biased'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/act_quant/conv1-layer/act_quant/min/local_step'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/act_quant/max'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/act_quant/min'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/bias'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/kernel'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/weights_quant/max'))
# print("conv1-layer/bias", model.get_variable_value('conv1-layer/weights_quant/min'))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '03_QAT_EMNIST_DEEP', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff731ab8fd0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph 

NanLossDuringTrainingError: NaN loss during training.

In [ ]:
def make_serving_input_receiver_fn():
    inputs = {'image': tf.placeholder(shape=[1,784], dtype=tf.float32, name='image')}
    return tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)

if tf.gfile.Exists(export_dir):
        tf.gfile.DeleteRecursively(export_dir)

model.export_savedmodel(export_dir_base=export_dir,serving_input_receiver_fn=make_serving_input_receiver_fn(),as_text=False)

In [ ]:
saved_model_dir = os.path.join(export_dir, os.listdir(export_dir)[-1]) 

predictor_fn = tf.contrib.predictor.from_saved_model(
  export_dir = saved_model_dir)



In [ ]:
35
dir = target + "/PNGs/"
pngFname = dir + random.choice(os.listdir(dir))

pngFile=open(pngFname,'rb')
png_string = pngFile.read();
pngFile.close()

#sample = tf.cast(tf.image.decode_png(png_string, channels=1), tf.int8)
sample = tf.cast(tf.image.decode_png(png_string, channels=1), tf.float32)

a = np.ones(shape=(28,28,1),dtype=np.float32)    

img = tf.reshape(sample, [28, 28, 1])

with tf.Session() as sess:
    a = img.eval()

b = np.ones(shape=(1,784),dtype=np.float32)

b = a.reshape((1,784))

output = predictor_fn({'image': b})
#print(output)
#print ("train_data.shape: " + str(sample.get_shape()))
#print ("train_data.shape: " + str(img.get_shape()))

max = 0

for x in range(10):
    if output['probabilities'][0][x] >= output['probabilities'][0][max]:
        max = x

f, ax = plt.subplots()
ax.imshow(a.reshape(img_shape), cmap='binary')
xlabel = "Result: Pred. No. : {0}".format(max)
ax.set_title(xlabel)


In [ ]:
# record_iterator = tf.python_io.tf_record_iterator(path=testing_recname)


# count = 0
# success = 0

# with tf.Session() as sess:
#     tf.train.start_queue_runners()

#     for string_record in record_iterator:
#         if count%100 == 0:
#             print("Success / Total: ", success, " / ", count, "Accuracy: {0:.2%}".format(success/(count+1)))

#         example = tf.train.Example()
#         example.ParseFromString(string_record)

#         sample = tf.cast(tf.image.decode_png(example.features.feature['image/encoded'].bytes_list.value[0], channels=1), tf.float32)
#         label = int(example.features.feature['image/class/label'].int64_list.value[0])

#         a = np.ones(shape=(28,28,1),dtype=np.float32)    

#         img = tf.reshape(sample, [28, 28, 1])

#         with sess.as_default():
#             a = img.eval()

#         b = np.ones(shape=(1,784),dtype=np.float32)

#         b = a.reshape((1,784))

#         output = predictor_fn({'image': b})

#         max = 0

#         for x in range(10):
#             if output['probabilities'][0][x] >= output['probabilities'][0][max]:
#                 max = x

#         if max == label:
#             success = success + 1

#         count = count + 1


In [ ]:
saved_model_dir = os.path.join(export_dir, os.listdir(export_dir)[-1]) 

converter = tf.contrib.lite.TFLiteConverter.from_saved_model(saved_model_dir)

converter.inference_type = tf.contrib.lite.constants.QUANTIZED_UINT8
# converter.inference_input_type = tf.contrib.lite.constants.FLOAT
# converter.output_format = tf.contrib.lite.constants.GRAPHVIZ_DOT
converter.output_format = tf.contrib.lite.constants.TFLITE
input_arrays = converter.get_input_arrays()
converter.quantized_input_stats = {input_arrays[0] : (0, 1)}  # mean, std_dev
converter.default_ranges_stats = (-127,127)

tflite_model = converter.convert()

if (converter.output_format == tf.contrib.lite.constants.TFLITE):
    open("converted_model.tflite", "wb").write(tflite_model)

if (converter.output_format == tf.contrib.lite.constants.GRAPHVIZ_DOT):
    open("converted_model.graphdot", "wb").write(tflite_model)

In [ ]:
interpreter = tf.contrib.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape']

########################################################

dir = target + "/PNGs/"
pngFname = dir + random.choice(os.listdir(dir))

pngFile=open(pngFname,'rb')
png_string = pngFile.read();
pngFile.close()

sample = tf.cast(tf.image.decode_png(png_string, channels=1), tf.uint8)

a = np.ones(shape=(28,28,1),dtype=np.uint8)    

img = tf.reshape(sample, [1, 784])

with tf.Session() as sess:
    a = img.eval()

b = np.ones(shape=input_shape,dtype=np.uint8)
b = a.reshape(input_shape)

########################################################

input_data = np.array(np.random.random_sample(input_shape), dtype=np.uint8)

with tf.Session() as sess:
    input_data = img.eval()

interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print("output_data: ", output_data)

f, ax = plt.subplots()
ax.imshow(input_data.reshape(img_shape), cmap='gray')
xlabel = "Result: Pred. No. : {0}".format(output_data)
ax.set_title(xlabel)


In [ ]:
record_iterator = tf.python_io.tf_record_iterator(path=testing_recname)
input_data = np.array(np.random.random_sample(input_shape), dtype=np.uint8)

interpreter.reset_all_variables()
interpreter = tf.contrib.lite.Interpreter(model_path="converted_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


count = 0
success = 0

for string_record in record_iterator:
    if count%1000 == 0:
        print("Success / Total: ", success, " / ", count, "Accuracy: {0:.2%}".format(success/(count+1)))

    example = tf.train.Example()
    example.ParseFromString(string_record)

    label = int(example.features.feature['image/class/label'].int64_list.value[0])

#   sample = tf.cast(tf.image.decode_png(example.features.feature['image/encoded'].bytes_list.value[0], channels=1), tf.uint8)
    x = np.frombuffer(example.features.feature['image/encoded'].bytes_list.value[0], dtype='uint8')
    sample = cv2.imdecode(x, cv2.IMREAD_UNCHANGED).reshape(1,784);

    interpreter.set_tensor(input_details[0]['index'], sample)

    interpreter.invoke()

    output_data = interpreter.get_tensor(output_details[0]['index'])

    if output_data[0][0] == label:
        success = success + 1

    count = count + 1

print("Final : Success / Total: ", success, " / ", count, "Accuracy: {0:.2%}".format(success/(count+1)))


In [ ]:
    keys_to_features = {
        "image/encoded":  tf.FixedLenFeature([], tf.string),
        "image/format":  tf.FixedLenFeature([], tf.string),
        "image/class/label": tf.FixedLenFeature([], tf.int64),
        "image/height": tf.FixedLenFeature([], tf.int64),
        "image/width": tf.FixedLenFeature([], tf.int64)
    }
